<a href="https://colab.research.google.com/github/lizawang/PCL/blob/main/pcl_linearmodel_with_augmenteddata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%ls


sample_data/


In [2]:
%cd drive/MyDrive/PCL/
%ls

/content/drive/MyDrive/PCL
10-langs-subtask1_all/            raw.pickle
5nn_predictions.csv               README.txt
backtranslation_subtask2/         rebuilddata.py
data1                             ref/
data1_augmented.csv               res/
data1_aug.pickle                  sentiment_scoring.py
data1.pickle                      task4_test.tsv
data2.pickle                      ten-langs-regex-cleaned-binary-label.csv
DeepMoji/                         thePoorerTheMerrier/
dev_semeval_parids-labels.csv     tptm.csv
dontpatronizeme_categories.tsv    train_semeval_parids-labels.csv
dontpatronizeme_pcl.tsv           trn_5nn_predictions.csv
dont_patronize_me.py              trndf1.csv
emoji_feature_vectors_trn_x1.csv  trndf2.csv
emoji_feature_vectors_tst_x1.csv  trn_encoded_with_senti_df1
evaluation.py                     trn_x1_augmented
model_graph.png                   trn_x1_original
modelsForStack/                   tstdf1.csv
pcl.ipynb                         tstdf2.csv
pcl-task1-aug

# This is for subtask 1

In [3]:
!pip install emoji
from DeepMoji.deepmoji.attlayer import AttentionWeightedAverage
from DeepMoji import main_emoji as emoji
from dont_patronize_me import DontPatronizeMe as dpm
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack, csr_matrix
from sklearn import svm
from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pickle
from sklearn.metrics import f1_score, classification_report

In [4]:
trn_file1 = "dontpatronizeme_pcl.tsv"
trn_file2 = "dontpatronizeme_categories.tsv"
tst_path = "task4_test.tsv"
dpm_o = dpm('.', tst_path)
dpm_o.load_task1()
dpm_o.load_task2()
dpm_o.load_test()
data1 = dpm_o.train_task1_df
data2 = dpm_o.train_task2_df
data_tst = dpm_o.test_set

Map of label to numerical label:
{'Unbalanced_power_relations': 0, 'Shallow_solution': 1, 'Presupposition': 2, 'Authority_voice': 3, 'Metaphors': 4, 'Compassion': 5, 'The_poorer_the_merrier': 6}


In [ ]:
# the final test set
data1_tst = np.array(data_tst)

In [ ]:
## this data1 is augmented
data1_aug = pd.read_csv("data1_augmented.csv", encoding="utf-8")
data1_aug.dropna(inplace=True)
trn_x1_all = data1_aug['text']
trn_txt_all = data1_aug[['text']]
trn_y1_all = data1_aug['label']
trn_x1_all

0        talking to the media after arriving at Islamab...
1        """" world record guinness of 540 pounds of 7 ...
2        " The second dear memory I cherished was the d...
3        """" today 's decision holds and affirms Ameri...
4        Leaving his prepared notes , Francis shared hi...
                               ...                        
19400    this helps to explain why many white working-c...
19401    The fate of nearly 790,000 undocumented young ...
19402    "" brixton music group recognizes that the sea...
19403    the seasiders crashed to a nine-shot defeat - ...
19404    The theme 'marche for our lives ' , the demons...
Name: text, Length: 19405, dtype: object

In [ ]:
"""
len([l for l in data1["label"] if l == 1])
data1_neg = data1[data1["label"]==0]
data1_neg = data1_neg[["text", "label"]]
data1_neg
"""

'\nlen([l for l in data1["label"] if l == 1])\ndata1_neg = data1[data1["label"]==0]\ndata1_neg = data1_neg[["text", "label"]]\ndata1_neg\n'

In [ ]:
"""
lang_list = ['sv', 'ru', 'pt','nl', 'it', 'fr', 'fi', 'es', 'de', 'cs']
data1_pos = None
for lan in lang_list:
  filepath = "10-langs-subtask1_all/%s.csv" % (lan)
  text = pd.read_csv(filepath, encoding="utf-8")
  text.columns = ["index", "text"]
  label = pd.DataFrame(np.ones(shape=(len(text),), dtype=np.int8))
  df = pd.concat([text["text"], label], axis = 1)
  df.columns = ["text", "label"]
  data1_pos = pd.concat([data1_pos, df], axis=0)

data1_augmented = pd.concat([data1_neg, data1_pos], axis=0) #row 7816 has empty text
data1_augmented = data1_augmented[data1_augmented["text"] != ""]
data1_augmented = data1_augmented.sample(frac=1) #shuffle rows for the new combined data
data1_augmented.to_csv("data1_augmented.csv", encoding="utf-8")
"""

'\nlang_list = [\'sv\', \'ru\', \'pt\',\'nl\', \'it\', \'fr\', \'fi\', \'es\', \'de\', \'cs\']\ndata1_pos = None\nfor lan in lang_list:\n  filepath = "10-langs-subtask1_all/%s.csv" % (lan)\n  text = pd.read_csv(filepath, encoding="utf-8")\n  text.columns = ["index", "text"]\n  label = pd.DataFrame(np.ones(shape=(len(text),), dtype=np.int8))\n  df = pd.concat([text["text"], label], axis = 1)\n  df.columns = ["text", "label"]\n  data1_pos = pd.concat([data1_pos, df], axis=0)\n\ndata1_augmented = pd.concat([data1_neg, data1_pos], axis=0) #row 7816 has empty text\ndata1_augmented = data1_augmented[data1_augmented["text"] != ""]\ndata1_augmented = data1_augmented.sample(frac=1) #shuffle rows for the new combined data\ndata1_augmented.to_csv("data1_augmented.csv", encoding="utf-8")\n'

In [5]:
data1 = data1[data1["text"] != ""] #remove the empty text row 8639 with label 0
for i, s in enumerate(data1["text"]):
  if s=="":
    print(i)

# Dump data1 and data2 into pickle files

In [ ]:
with open("data2.pickle", "wb") as f:
  pickle.dump(data2, f)

In [ ]:
with open("data1.pickle", "wb") as fi:
  pickle.dump(data1, fi)

# Get test set from original and training set from augmented data

In [6]:
## get test set from the origninal dataset
x1, y1 = data1[["text"]], data1["label"]
_, tst_x1, _, tst_y1 = train_test_split(x1, y1, test_size=0.2, random_state=36)
tst_txt_df = tst_x1
tst_x1 = tst_x1['text']
tst_x1

4988     In fact , romance by far out-produces and out-...
3053     In the past , Dorel has had moments of brillia...
1984     A man who reached out to a community Facebook ...
3547     Other westerners who met similar fate to Foley...
4891     "The home affairs department must explain why ...
                               ...                        
7193     Morris , who is blind , made the call in the S...
8843     He said they are not always able to transfer m...
6110     """ The broad part of officials and the masses...
5848     It seems some Chinese immigrants come across a...
10172    "Meanwhile "" throughout this island , the hig...
Name: text, Length: 2094, dtype: object

In [7]:
## get the training set from the augmented dataset
trndf1 = pd.read_csv("ten-langs-regex-cleaned-binary-label.csv", encoding="utf-8")
trndf1 = trndf1.dropna() #drop the nan sentence on 733 row
trndf1 = trndf1.sample(frac=1)
trndf1

,text,label
6262,it was a tridional n leage budget . pro-elite ...,0
9499,"In their tweet , Reham stressed how a "" real l...",1
1001,the letter came as the country grapples with a...,0
9188,The Word of God is truth that 's life and able...,1
14445,"Under the pretext of """" to know how the other ...",1
...,...,...
5884,new york ( afp ) - the number of homeless scho...,0
12889,But this can be devastating for disabled and e...,1
4224,the bank group 's support focused on major tra...,0
8670,When travelling through different communities ...,1


In [8]:
trn_x1 = trndf1['text']
trn_txt_df = trndf1[['text']]
trn_y1 = trndf1["label"]
trn_txt_df

,text
6262,it was a tridional n leage budget . pro-elite ...
9499,"In their tweet , Reham stressed how a "" real l..."
1001,the letter came as the country grapples with a...
9188,The Word of God is truth that 's life and able...
14445,"Under the pretext of """" to know how the other ..."
...,...
5884,new york ( afp ) - the number of homeless scho...
12889,But this can be devastating for disabled and e...
4224,the bank group 's support focused on major tra...
8670,When travelling through different communities ...


In [9]:
for i, t in enumerate(trn_x1): ## check if 733 of trn_x1 is still nan
  if t is np.nan:
    print(i, t)

In [ ]:
#trn_x1.to_csv("trn_x1_augmented", encoding="utf-8")

* **Add** both word and charactar ngrams using **TF-IDF** vectorizer

In [10]:
def encoding(trn_x, tst_x=None, ngram = (1, 3), analyzer = "word", lowercase=True):
  vectorizer = TfidfVectorizer(strip_accents="unicode", stop_words="english", lowercase=lowercase,
                               analyzer=analyzer,ngram_range=ngram)
  trn_x_encoded = vectorizer.fit_transform(trn_x)
  if tst_x is not None:
    tst_x_encoded = vectorizer.transform(tst_x)
  else:
    tst_x_encoded = ""
  return trn_x_encoded, tst_x_encoded


In [12]:
# slipted data
trn_x_encoded_word, tst_x_encoded_word = encoding(trn_x1, tst_x1, lowercase=True)
trn_x_encoded_char, tst_x_encoded_char = encoding(trn_x1, tst_x1, analyzer="char", lowercase=True)
trn_x_encoded1 = hstack((trn_x_encoded_word, trn_x_encoded_char))
tst_x_encoded1 = hstack((tst_x_encoded_word, tst_x_encoded_char))
tst_x_encoded1

<2094x454621 sparse matrix of type '<class 'numpy.float64'>'
	with 815699 stored elements in COOrdinate format>

In [ ]:
# data1_all augmented
trn_x_encoded_word, tst_x_encoded_word = encoding(trn_x1_all, data1_tst, lowercase=True)
trn_x_encoded_char, tst_x_encoded_char = encoding(trn_x1_all, data1_tst, analyzer="char", lowercase=False)
trn_x1_all_encoded = hstack((trn_x_encoded_word, trn_x_encoded_char))
tst_x1_all_encoded = hstack((tst_x_encoded_word, tst_x_encoded_char))
tst_x1_all_encoded

<3828x574844 sparse matrix of type '<class 'numpy.float64'>'
	with 1481680 stored elements in COOrdinate format>

In [13]:
trn_x_encoded1

<15155x454621 sparse matrix of type '<class 'numpy.float64'>'
	with 6444530 stored elements in COOrdinate format>

### **Add** sentiment scores using ruled based pretrained model **VADER** or machine-learning pretrained model **Flair**

In [ ]:
!pip install vaderSentiment
!pip install flair
from sentiment_scoring import senti_score

In [15]:
# splited trn
sc_trn1 = np.array(senti_score(trn_x1, "vader"))
sc_tst1 = np.array(senti_score(tst_x1, "vader"))
trn_x_encoded_with_senti = hstack((trn_x_encoded1, csr_matrix(sc_trn1)))
tst_x_encoded_with_senti = hstack((tst_x_encoded1, csr_matrix(sc_tst1)))
sc_trn1

array([[ 0.021 ,  0.862 ,  0.117 ,  0.8403],
       [ 0.114 ,  0.762 ,  0.124 ,  0.0516],
       [ 0.    ,  1.    ,  0.    ,  0.    ],
       ...,
       [ 0.064 ,  0.725 ,  0.211 ,  0.7269],
       [ 0.138 ,  0.813 ,  0.049 , -0.786 ],
       [ 0.    ,  0.876 ,  0.124 ,  0.6597]])

In [ ]:
# for all trn
sc_trn1 = np.array(senti_score(trn_x1_all, "vader"))
sc_tst1 = np.array(senti_score(data1_tst, "vader"))
trn_x1_all_encoded_with_senti = hstack((trn_x1_all_encoded, csr_matrix(sc_trn1)))
tst_x1_all_encoded_with_senti = hstack((tst_x1_all_encoded, csr_matrix(sc_tst1)))
sc_trn1[0], csr_matrix(sc_trn1).toarray()[0]

([0.041, 0.73, 0.229, 0.8779], array([0.041 , 0.73  , 0.229 , 0.8779]))

In [18]:
trn_x_encoded1, tst_x_encoded1, trn_x_encoded_with_senti, tst_x_encoded_with_senti

(<15155x457427 sparse matrix of type '<class 'numpy.float64'>'
 	with 6489351 stored elements in COOrdinate format>,
 <2094x457427 sparse matrix of type '<class 'numpy.float64'>'
 	with 836415 stored elements in COOrdinate format>,
 <15155x457431 sparse matrix of type '<class 'numpy.float64'>'
 	with 6540562 stored elements in COOrdinate format>,
 <2094x457431 sparse matrix of type '<class 'numpy.float64'>'
 	with 843399 stored elements in COOrdinate format>)

In [ ]:
## write into files so no need to run it again next time
#trn_encoded_with_senti_df = pd.DataFrame.sparse.from_spmatrix(trn_x_encoded_with_senti)
#tst_encoded_with_senti_df = pd.DataFrame.sparse.from_spmatrix(tst_x_encoded_with_senti)
#trn_encoded_with_senti_df.to_csv("trn_encoded_with_senti_df1")
#tst_encoded_with_senti_df.to_csv("tst_encoded_with_senti_df1")

### Add **emoji prediction vectors** into the feature vectors

In [16]:
max_len = 80

In [17]:
# produce prediction top5 from the function
emoji_df1_trn = pd.DataFrame(emoji.emoji_prediction(trn_txt_df, maxlen=max_len))
emoji_df1_trn = pd.DataFrame(emoji.emoji_prediction(trn_txt_df, maxlen=max_len))

Tokenizing using dictionary from /content/drive/MyDrive/PCL/DeepMoji/model/vocabulary.json
Loading model from /content/drive/MyDrive/PCL/DeepMoji/model/deepmoji_weights.hdf5.
Model: "DeepMoji"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 80, 256)      12800000    ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 80, 256)      0           ['embedding[0][0]']              
                                                                                                  
 bi_lstm_0 (Bid

In [ ]:
emoji_df1_tst = pd.DataFrame(emoji.emoji_prediction(tst_txt_df, maxlen=max_len))

In [ ]:
emoji_trnx1 =  emoji_df1_trn[emoji_df1_trn.columns[-64:]]
emoji_tstx1 = emoji_df1_tst[emoji_df1_tst.columns[-64:]]

In [ ]:
emoji_trnx1

,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75
0,0.003902,0.003144,0.002172,0.007684,0.003493,0.008043,0.012574,0.011613,0.031763,0.002523,0.003797,0.018456,0.005843,0.030239,0.000657,0.004263,0.012688,0.060602,0.007520,0.005722,0.004437,0.115412,0.009594,0.004561,0.055930,0.011705,0.001510,0.013387,0.003931,0.002887,0.019267,0.013441,0.008764,0.056443,0.050920,0.008489,0.001837,0.004157,0.003714,0.002473,0.062329,0.012448,0.007450,0.004834,0.006939,0.010322,0.051934,0.034430,0.005517,0.002288,0.010055,0.005820,0.006401,0.016319,0.008308,0.006732,0.002145,0.009100,0.018133,0.018144,0.013681,0.039780,0.006316,0.011016
1,0.005127,0.013586,0.002661,0.003350,0.000904,0.012995,0.005279,0.006781,0.012200,0.001908,0.004423,0.013459,0.020442,0.002971,0.001383,0.001835,0.009044,0.004689,0.003047,0.024727,0.005838,0.020251,0.049059,0.001178,0.035783,0.033984,0.001785,0.030399,0.006533,0.003556,0.012235,0.006284,0.134241,0.030279,0.048326,0.009554,0.001529,0.021561,0.004917,0.011859,0.013490,0.008717,0.006270,0.020887,0.010928,0.016790,0.036766,0.022239,0.002545,0.004012,0.008158,0.003557,0.022341,0.017795,0.011191,0.115052,0.004575,0.002411,0.011822,0.007022,0.006179,0.007381,0.015914,0.003997
2,0.001358,0.007325,0.002164,0.001603,0.000273,0.010033,0.003272,0.003894,0.005593,0.001050,0.002546,0.001849,0.012400,0.004989,0.001474,0.001100,0.002544,0.003780,0.000768,0.015351,0.003260,0.023862,0.035311,0.000411,0.007628,0.023908,0.000500,0.031092,0.002610,0.003424,0.005522,0.004240,0.299341,0.031532,0.036681,0.008280,0.000599,0.034092,0.001423,0.005277,0.009598,0.003416,0.017372,0.018462,0.033125,0.016628,0.022427,0.008393,0.000572,0.000919,0.002958,0.005249,0.020868,0.005983,0.003698,0.132550,0.002270,0.006710,0.014621,0.005408,0.001927,0.008010,0.015015,0.001460
3,0.000907,0.006270,0.001169,0.002423,0.000211,0.016971,0.002082,0.001641,0.022318,0.000534,0.000747,0.001530,0.003538,0.007937,0.000845,0.001077,0.003117,0.005327,0.002665,0.007663,0.001208,0.117611,0.021668,0.000974,0.090199,0.011319,0.000281,0.032096,0.000911,0.001459,0.007443,0.001389,0.132410,0.008047,0.078747,0.015283,0.000235,0.019441,0.001940,0.002919,0.008968,0.002832,0.004239,0.012180,0.012739,0.015900,0.107560,0.044752,0.000486,0.000665,0.001924,0.001760,0.009143,0.002720,0.002171,0.087614,0.002556,0.002667,0.010988,0.007887,0.006320,0.013885,0.003157,0.002335
4,0.002756,0.012141,0.002344,0.003331,0.000675,0.013813,0.005703,0.005601,0.011995,0.002461,0.003046,0.006338,0.006496,0.012066,0.001958,0.002757,0.006808,0.012500,0.001505,0.020849,0.005074,0.045914,0.030738,0.001624,0.027136,0.030610,0.001415,0.022971,0.002307,0.003381,0.018705,0.008074,0.125535,0.024634,0.043456,0.010554,0.001043,0.028283,0.007023,0.005097,0.020802,0.008330,0.013230,0.014753,0.037309,0.018985,0.049381,0.020670,0.001697,0.001956,0.012121,0.004628,0.015705,0.009923,0.010668,0.097144,0.013821,0.006033,0.026154,0.007259,0.003358,0.012590,0.010354,0.004409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15150,0.003458,0.003411,0.002050,0.004504,0.001611,0.018252,0.009451,0.022500,0.021572,0.005467,0.009126,0.010067,0.006015,0.014631,0.001701,0.011801,0.020654,0.012789,0.007399,0.004978,0.009404,0.058878,0.020302,0.003009,0.041312,0.011554,0.002573,0.025373,0.003616,0.002775,0.029119,0.017634,0.008660,0.046545,0.042496,0.015981,0.002655,0.005565,0.003934,0.001818,0.013980,0.009776,0.003518,0.014940,0.059256,0.018743,0.049617,0.038806,0.003429,0.003140,0.036353,0.004109,0.010119,0.024656,0.013992,0.007664,0.001887,0.026209,0.021205,0.018411,0.009747,0.028816,0.010205,0.026782
15151,0.010864,0.007201,0.008702,0.015379,0.002119,0.037926,0.010779,0.0049

###### Or read emoji predictions from the saved files with unshuffled trn_df1

In [ ]:
emoji_df1_trn = pd.read_csv("emoji_feature_vectors_trn_x1.csv", encoding="utf-8")
emoji_trnx1 =  emoji_df1_trn[emoji_df1_trn.columns[-64:]]
emoji_df1_tst = pd.read_csv("emoji_feature_vectors_tst_x1.csv", encoding="utf-8")
emoji_tstx1 = emoji_df1_tst[emoji_df1_tst.columns[-64:]]
emoji_trnx1

##### add the emoji vectors to trn and tst vectors

In [ ]:
# add the emoji vectors to trn and tst vectors
trn_x_encoded_with_emoji = hstack((trn_x_encoded1, csr_matrix(emoji_trnx1)))
tst_x_encoded_with_emoji = hstack((tst_x_encoded1, csr_matrix(emoji_tstx1)))
trn_x_encoded_with_emoji, tst_x_encoded_with_emoji.toarray()

### Get labels ready

In [ ]:
trn_y_encoded1 = np.array(trn_y1)
tst_y_encoded1 = np.array(tst_y1)
tst_y_encoded1

### Train svm model without any sementic scores

In [ ]:
model1 = svm.SVC(kernel="linear", C=1, gamma="scale")
model1.fit(trn_x_encoded1, trn_y_encoded1)

SVC(C=1, kernel='linear')

In [ ]:
pred1 = model1.predict(tst_x_encoded1)

In [ ]:
pred1

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
f1_sc1 = f1_score(tst_y_encoded1, pred1)
cls_report1 = classification_report(tst_y_encoded1, pred1)
f1_sc1, print(cls_report1) #0.694 macro, 0.505 binary

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1893
           1       0.54      0.48      0.51       201

    accuracy                           0.91      2094
   macro avg       0.74      0.72      0.73      2094
weighted avg       0.91      0.91      0.91      2094



(0.5052631578947369, None)

### Train svm with sentiment scores

In [ ]:
model_with_senti = svm.SVC(kernel="linear", C=1, gamma="scale")
model_with_senti.fit(trn_x_encoded_with_senti, trn_y_encoded1)

SVC(C=1, kernel='linear')

In [ ]:
pred_with_senti = model_with_senti.predict(tst_x_encoded_with_senti) # should predict encoded matrix or just the text strings?

In [ ]:
f1_sc_with_senti = f1_score(tst_y_encoded1, pred_with_senti)
cls_report_senti = classification_report(tst_y_encoded1, pred_with_senti)
f1_sc_with_senti, print(cls_report_senti) #0.7076 macro, 0.521 binary

### Train svm model with emoji scores

In [ ]:
model_with_emoji = svm.SVC(kernel="linear", C=1, gamma="scale")

In [ ]:
model_with_emoji.fit(trn_x_encoded_with_emoji, trn_y_encoded1)

In [ ]:
pred_with_emoji = model_with_emoji.predict(tst_x_encoded_with_emoji)

In [ ]:
f1_sc_with_emoji =  f1_score(tst_y_encoded1, pred_with_emoji)
cls_report_emoji = classification_report(tst_y_encoded1, pred_with_emoji)
f1_sc_with_emoji, print(cls_report_emoji) #0.4986 binary

## Ensemble Models

In [ ]:
!pip install vecstack
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from vecstack import stacking

##### Stacking models

In [ ]:
## models for stacking
base1 = RandomForestClassifier(random_state=0, max_features='auto', max_depth=3)
base2 = LogisticRegression(solver='liblinear', max_iter=200)
pipeline = make_pipeline(StandardScaler(with_mean=False),
                           svm.SVC(C=1.0, kernel='linear'))
svc = svm.SVC(C=1.0, kernel='linear')

##### stack with both emoji and senti

In [ ]:
# stacking with emoji and senti
# get the prediction for both trn_x and tst_x
def ensemble_stacking_both_fit(trn_x, trn_y, tst_x):
  trn_x_s, tst_x_s = stacking([base1, base3], trn_x, trn_y, tst_x, stratified=True,
                              regression=False, n_folds=5,
                              shuffle=True, verbose=2)
  print(tst_x_s.shape)
  # stack the prediciton from emoji to the trn_x_s and tst_x_s
  trn_x_s = np.hstack((np.array(emoji_trnx1), trn_x_s))
  tst_x_s = np.hstack((np.array(emoji_tstx1), tst_x_s))
  meta_model = base2
  meta_model.fit(trn_x_s, trn_y)
  pred = meta_model.predict(tst_x_s)
  return pred


In [ ]:
pred_stacking_both = ensemble_stacking_both_fit(trn_x_encoded_with_senti, 
                                               trn_y_encoded1,
                                               tst_x_encoded_with_senti)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.74331904]
    fold  1:  [0.75882547]
    fold  2:  [0.74826790]
    fold  3:  [0.74661828]
    fold  4:  [0.78686902]
    ----
    MEAN:     [0.75677994] + [0.01591507]
    FULL:     [0.75677994]

model  1:     [SVC]
    fold  0:  [0.98152425]
    fold  1:  [0.98449357]
    fold  2:  [0.98152425]
    fold  3:  [0.98317387]
    fold  4:  [0.98317387]
    ----
    MEAN:     [0.98277796] + [0.00113140]
    FULL:     [0.98277796]

(15155, 2)


In [ ]:
f1_stacking_both = f1_score(tst_y_encoded1, pred_stacking_both)
cls_report_stacking_both = classification_report(tst_y_encoded1, pred_stacking_both)
print(cls_report_stacking_both, f1_stacking_both)
# label:1       0.54      0.50      0.52;  binary f1: 0.518   

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1893
           1       0.54      0.50      0.52       201

    accuracy                           0.91      2094
   macro avg       0.74      0.73      0.73      2094
weighted avg       0.91      0.91      0.91      2094
 0.5181347150259068


##### Stack the predictions from 5 nn models on top of emoji and senti

In [ ]:
### read the predictions of nn from the files nn5_preds
tst_nn_preds = pd.read_csv("5nn_predictions.csv", encoding="utf-8")
tst_nn_preds = np.array(tst_nn_preds)
trn_nn_preds = pd.read_csv("trn_5nn_predictions.csv", encoding="utf-8")
trn_nn_preds = np.array(trn_nn_preds)

In [ ]:
trn_nn_preds

array([[0.0000000e+00, 5.4094577e-01, 2.9859180e-01, 2.8656210e-01,
        2.3361695e-01, 2.4493778e-01],
       [1.0000000e+00, 5.4094577e-01, 5.7595766e-01, 6.0827285e-01,
        5.7385770e-01, 5.7605040e-01],
       [2.0000000e+00, 5.4094577e-01, 4.4539046e-01, 4.6011204e-01,
        3.2503462e-01, 3.7752926e-01],
       ...,
       [1.5152000e+04, 5.4094577e-01, 6.1489700e-01, 6.3552010e-01,
        6.0550475e-01, 6.2028307e-01],
       [1.5153000e+04, 5.4094577e-01, 3.0497026e-01, 4.0300885e-01,
        3.1272823e-01, 9.6421397e-01],
       [1.5154000e+04, 9.9765170e-01, 6.1489700e-01, 6.3656425e-01,
        5.7576823e-01, 6.2028307e-01]])

In [ ]:
# stacking with emoji and senti and 5 nn models predictions
# get the predictions for both trn_x and tst_x
def ensemble_stacking_nn_fit(trn_x, trn_y, tst_x):
  trn_x_s, tst_x_s = stacking([base1, svc], trn_x, trn_y, tst_x, stratified=True,
                              regression=False, n_folds=5,
                              shuffle=True, verbose=2)
  print(tst_x_s.shape)
  # stack the senti scores 
  trn_x_s = np.hstack((sc_trn1, trn_x_s))
  tst_x_s = np.hstack((sc_tst1, tst_x_s))
  # stack the prediciton from emoji preds to the trn_x_s and tst_x_s
  trn_x_s = np.hstack((np.array(emoji_trnx1), trn_x_s))
  tst_x_s = np.hstack((np.array(emoji_tstx1), tst_x_s))
  # stack the predictions from the 5nn models to the trn_x_s and tst_x_s
  #trn_x_s = np.hstack((trn_x_s, trn_nn_preds))
  #tst_x_s = np.hstack((tst_x_s, tst_nn_preds))
  # scale the data
  #ssl = StandardScaler()
  #trn_x_s = ssl.fit_transform(trn_x_s)
  #tst_x_s = ssl.transform(tst_x_s)
  print("tst_x_s:{}".format(tst_x_s))
  meta_model = base2
  meta_model.fit(trn_x_s, trn_y)
  pred = meta_model.predict(tst_x_s)
  return pred

In [ ]:
pred_stacking_nn = ensemble_stacking_nn_fit(trn_x_encoded1, 
                                               trn_y_encoded1,
                                               tst_x_encoded1)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.83767733]
    fold  1:  [0.82612999]
    fold  2:  [0.84295612]
    fold  3:  [0.84757506]
    fold  4:  [0.84460574]
    ----
    MEAN:     [0.83978885] + [0.00754827]
    FULL:     [0.83978885]

model  1:     [SVC]


In [ ]:
f1_stacking_nn = f1_score(tst_y_encoded1, pred_stacking_nn)
cls_report_stacking_nn = classification_report(tst_y_encoded1, pred_stacking_nn)
print(cls_report_stacking_nn, f1_stacking_nn) #1       0.54      0.50      0.52  , f1_binary: 0.5206 with scalar

In [ ]:
print(pred_stacking_nn, len(pred_stacking_nn))

[0 0 0 ... 0 0 1] 2094


##### stack either with senti or emoji to the data in the beginning

In [ ]:
# only with either emoji or senti
model_stacking = StackingClassifier(estimators=[('rf', base1), ('svc', base3)], 
                                    final_estimator=base2, passthrough=True)
model_stacking.fit(trn_x_encoded_with_senti, trn_y_encoded1)

In [ ]:
pred_stk = model_stacking.predict(tst_x_encoded_with_senti)
f1_stk = f1_score(tst_y_encoded1, pred_stk)
cls_report_stk = classification_report(tst_y_encoded1, pred_stk)
f1_stk, print(cls_report_stk) #0.37 emoji with passthrough=True,

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1893
           1       0.68      0.26      0.37       201

    accuracy                           0.92      2094
   macro avg       0.80      0.62      0.66      2094
weighted avg       0.90      0.92      0.90      2094



(0.37410071942446044, None)

#### bagging or boosting models

##### bagging models

In [ ]:
def ensemble_model(classifier_name, base_estimator=None, estimator_n=10):
  if classifier_name == "bagging" and base_estimator is not None:
    model = BaggingClassifier(base_estimator, n_estimators=estimator_n,
                              max_samples=0.5, max_features=1)
  elif classifier_name == "boosting":
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.01)
  else:
    print("The classifier names can only be 'bagging' or 'boosting'.")
  return model

In [ ]:
# bagging ensemble 

# with senti
base_estm = svm.SVC(C=1.0, kernel='linear')
model_bagging = ensemble_model("bagging", base_estm)
model_bagging.fit(trn_x_encoded_with_senti, trn_y_encoded1)

BaggingClassifier(base_estimator=SVC(kernel='linear'), max_features=1,
                  max_samples=0.5)

In [ ]:
pred_bagging = model_bagging.predict(tst_x_encoded_with_senti)
f1_bag = f1_score(tst_y_encoded1, pred_bagging, average='binary')
cls_report_bag = classification_report(tst_y_encoded1, pred_bagging)
f1_bag, print(cls_report_bag) #with senti

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1893
           1       0.10      1.00      0.18       201

    accuracy                           0.10      2094
   macro avg       0.05      0.50      0.09      2094
weighted avg       0.01      0.10      0.02      2094



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.17516339869281047, None)

In [ ]:
# with emoji
base_estm = svm.SVC(C=1.0, kernel='linear')
model_bagging = ensemble_model("bagging", base_estm, estimator_n=20)
model_bagging.fit(trn_x_encoded_with_emoji, trn_y_encoded1)

BaggingClassifier(base_estimator=SVC(kernel='linear'), max_features=1,
                  max_samples=0.5, n_estimators=20)

In [ ]:
pred_bagging = model_bagging.predict(tst_x_encoded_with_emoji)
f1_bag = f1_score(tst_y_encoded1, pred_bagging, average='binary')
cls_report_bag = classification_report(tst_y_encoded1, pred_bagging)
f1_bag, print(cls_report_bag) #with emoji 0.49 with feature_n 0.5, set feature_n to 1.0 gets 0.515

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1893
           1       0.00      0.00      0.00       201

    accuracy                           0.90      2094
   macro avg       0.45      0.50      0.47      2094
weighted avg       0.82      0.90      0.86      2094



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.0, None)

##### boosting models

In [ ]:
# boosting ensemble
model_boosting = ensemble_model("boosting")
model_boosting.fit(trn_x_encoded_with_emoji, trn_y_encoded1)

GradientBoostingClassifier()

In [ ]:
pred_boosting = model_boosting.predict(tst_x_encoded_with_emoji)
print(pred_boosting)
cls_report_bst = classification_report(tst_y_encoded1, pred_boosting)
print(cls_report_bst)

[0 0 1 ... 0 0 0]
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      1893
           1       0.41      0.28      0.34       201

    accuracy                           0.89      2094
   macro avg       0.67      0.62      0.64      2094
weighted avg       0.88      0.89      0.88      2094



# **Fine-Tuning** with GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:

parameters = {'kernel': ('linear', 'rbf'), 'C':[0.01, 0.1, 1, 10, 100], 'gamma':('scale', 'auto'), 
              'class_weight': ['balanced']}
svc = svm.SVC()
gs = GridSearchCV(svc, parameters)
gs.fit(trn_x_encoded_with_senti, trn_y_encoded1)

In [ ]:
gs.cv_results_

In [ ]:
gs.best_params_

In [ ]:
pred_tuned = gs.predict(tst_x_encoded_with_senti) 
f1_sc_tuned = f1_score(tst_y_encoded1, pred_tuned, average="binary")
cls_report_tuned = classification_report(tst_y_encoded1, pred_tuned)

In [ ]:
f1_sc_tuned, print(cls_report_tuned)